### Mp3 File Name

In [8]:
import pandas as pd
import numpy as np
import json
import re
import glob
import os

In [2]:
df_master= pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Text/Result/10-Web Scrapping Data To Master/Arabic_Buckwalter_All_Deployment_Master Original.xlsx")
df_mp3 = df_master[["page","surah","ayah","surah latin"]]
df_mp3

,page,surah,ayah,surah latin
0,1,1,1,Al-Fatiha
1,1,1,1,Al-Fatiha
2,1,1,1,Al-Fatiha
3,1,1,1,Al-Fatiha
4,1,1,2,Al-Fatiha
...,...,...,...,...
78244,604,114,5,An-Nas
78245,604,114,5,An-Nas
78246,604,114,6,An-Nas
78247,604,114,6,An-Nas


In [3]:
file_name_list = []
for i in range(len(df_mp3)):
    surah = df_mp3.loc[i,"surah"]
    ayah = df_mp3.loc[i,"ayah"]
    file_name = f"{surah:03d}"+f"{ayah:03d}.mp3"
    file_name_list.append(file_name)
df_mp3["mp3_file"] = pd.Series(file_name_list)
df_mp3

<ipython-input-3-6e46b9e1febe>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mp3["mp3_file"] = pd.Series(file_name_list)


,page,surah,ayah,surah latin,mp3_file
0,1,1,1,Al-Fatiha,001001.mp3
1,1,1,1,Al-Fatiha,001001.mp3
2,1,1,1,Al-Fatiha,001001.mp3
3,1,1,1,Al-Fatiha,001001.mp3
4,1,1,2,Al-Fatiha,001002.mp3
...,...,...,...,...,...
78244,604,114,5,An-Nas,114005.mp3
78245,604,114,5,An-Nas,114005.mp3
78246,604,114,6,An-Nas,114006.mp3
78247,604,114,6,An-Nas,114006.mp3


In [4]:
df_mp3 = df_mp3[~(df_mp3.ayah == 0)]
df_mp3 

,page,surah,ayah,surah latin,mp3_file
0,1,1,1,Al-Fatiha,001001.mp3
1,1,1,1,Al-Fatiha,001001.mp3
2,1,1,1,Al-Fatiha,001001.mp3
3,1,1,1,Al-Fatiha,001001.mp3
4,1,1,2,Al-Fatiha,001002.mp3
...,...,...,...,...,...
78244,604,114,5,An-Nas,114005.mp3
78245,604,114,5,An-Nas,114005.mp3
78246,604,114,6,An-Nas,114006.mp3
78247,604,114,6,An-Nas,114006.mp3


In [20]:
df_page_first_mp3 = df_mp3.groupby("page")["page","mp3_file"].head(1)
df_page_first_mp3.rename(columns={"mp3_file":"first_mp3"}, inplace=True)
df_page_last_mp3 = df_mp3.groupby("page")["page","mp3_file"].tail(1)
df_page_last_mp3.rename(columns={"mp3_file":"last_mp3"}, inplace=True)

<ipython-input-20-73e64911cd0e>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_page_first_mp3 = df_mp3.groupby("page")["page","mp3_file"].head(1)
<ipython-input-20-73e64911cd0e>:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_page_last_mp3 = df_mp3.groupby("page")["page","mp3_file"].tail(1)


In [21]:
df_page_first_last_mp3 = pd.merge(df_page_first_mp3, df_page_last_mp3, how="inner", on="page")
df_page_first_last_mp3

,page,first_mp3,last_mp3
0,1,001001.mp3,001007.mp3
1,2,002001.mp3,002005.mp3
2,3,002006.mp3,002016.mp3
3,4,002017.mp3,002024.mp3
4,5,002025.mp3,002029.mp3
...,...,...,...
599,600,100010.mp3,102008.mp3
600,601,103001.mp3,105005.mp3
601,602,106001.mp3,108003.mp3
602,603,109001.mp3,111005.mp3


In [29]:
page_list = []
for i in range(1,10):
    df_var = df_mp3[df_mp3["page"] == i]
    #order_list = df_var["mp3_file"].to_list()
    order_list = list(df_var["mp3_file"].unique())
    surah = list(df_var["surah"].unique())
    surah_list = []
    for j in surah:
        df_var2 = df_var[df_var["surah"] == j]
        surah_latin = df_var2["surah latin"].unique()[0]
        #ayahs = df_var2["mp3_file"].to_list()
        ayahs = list(df_var2["mp3_file"].unique())
        surah_dict = {"surah":int(j),"surah-latin":surah_latin,"ayahs-in-page":ayahs}
        surah_list.append(surah_dict) 
    dict_var = {"page":i,"orderedAyahs":order_list,"surahs":surah_list}
    page_list.append(dict_var) 

In [17]:
#page_num = 1
#for i in page_list:
#    with open(f"index{page_num}.json","w") as file:
#        json.dump(i, file, indent=4)    
#    page_num += 1

In [12]:
#page_num = 1
#for i in page_list:
#    parent_dir = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Text/Result/11-Deployment File Mp3 Name/MP3 Files/husary/"
#    directory = f"Page {page_num:03d}"    
#    path = os.path.join(parent_dir, directory)
#    os.mkdir(path)
#    with open(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Text/Result/11-Deployment File Mp3 Name/MP3 Files/husary/Page {page_num:03d}/index.json","w") as file:
#        json.dump(i, file, indent=4)    
#    page_num += 1

In [31]:
page_num = 1
for i in page_list:
    parent_dir = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Text/Result/11-Deployment File Mp3 Name/MP3 Files/husary/"
    directory = f"Page {page_num:03d}"    
    path = os.path.join(parent_dir, directory)
    os.mkdir(path)
    with open(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Text/Result/11-Deployment File Mp3 Name/MP3 Files/husary/Page {page_num:03d}/index.json","w") as file:
        json.dump(i, file, indent=4)
    df_var = df_page_first_last_mp3[df_page_first_last_mp3["page"] == page_num]
    first = df_var.iloc[0,1]
    last = df_var.iloc[0,2]
    with open(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Text/Result/11-Deployment File Mp3 Name/MP3 Files/husary/Page {page_num:03d}/MP3 Files [{first}] - [{last}].txt","w") as file:
        file.write(f"{first};{last}")     
    page_num += 1